In [18]:
import numpy as np
import pandas as pd
import re
import os
from itertools import chain

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation, NMF
from sklearn.svm import LinearSVC
from sklearn.metrics import f1_score, accuracy_score
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import GridSearchCV, PredefinedSplit

from scipy.stats import uniform, randint


In [79]:
workPath = "workset/"

## skipPages is an optional parameter, which can be included
## if desired. It specifies pages that we should not even 
## consider when looking for riddles, in case someone spends
## time manually checking.
##
## skipPages should be filled with entries as follows:
## {"htID (should be the name of the folder)":pages to skip,
## "next htID":pages}
## Pages to skip will be represented as a list of ranges and
## integers.
## In order to represent a range of pages, you want to use 
## range(start, stop). Range is exclusive at the end, and
## indexing starts at 0, whereas page numbers start at 1, so 
## if you want to skip pages 1 through 10, you should put
## range(0, 11), if you want a single page, you can just write
## the number (page number - 1). It will simply ignore other 
## entries than integers and ranges

skipPages={"ht1":[range(0, 5), 7]}

for f in os.scandir(workPath):
    
    ## Within our workset, find folders (since volumes are imported as folders)
    if f.is_dir():
        
        ## Filter files in the folder for those ending in .txt and sort into
        ## alphabetical and numerical order, ascending. This allows us to stop
        ## looking once we get past the page which we think starts the end of
        ## the riddles, such as those containing "published" or related words
        skipList = range(0)
        for entry in skipPages.get(f.name, []):
            if type(entry) is range:
                skipList = chain(skipList, entry)
            elif type(entry) is int:
                skipList = chain(skipList, range(entry, entry+1))
        
        pageList = [p for p in sorted(os.scandir(workPath + f.name), key=lambda file: file.name) if p.name.endswith(".txt")]
        for i, page in enumerate(pageList):
            if i not in skipList:
                print(i)

[p for p in os.scandir(workPath + "ht1") if p.name.endswith(".txt")]

#print([f for f in os.scandir() if f.is_dir()])

5
6
7


[<DirEntry '8.txt'>,
 <DirEntry '5.txt'>,
 <DirEntry '4.txt'>,
 <DirEntry '6.txt'>,
 <DirEntry '7.txt'>,
 <DirEntry '3.txt'>,
 <DirEntry '2.txt'>,
 <DirEntry '1.txt'>]

In [14]:
## Will take in the pathway of your workset folder and return 
## a pandas dataframe attempting to separate riddles into
## different rows. 
## 
## If you happen to know what page of the book
def createDataSet(path, skipPages=None):
    return None


## Function called in createDataSet, which takes in the text
## of one page and returns riddles found on the page as a list
## of strings, or None if none are found
def getRiddles(page, lookForIntro, lookForConclusion):
    return page



    

[<DirEntry '002_Summary of Proposal_18h24.docx'>, <DirEntry 'Worklog11-25.jpg'>, <DirEntry 'RA Task List.docx'>, <DirEntry 'To Do List.docx'>, <DirEntry '.DS_Store'>, <DirEntry 'UofT_Fisher MSS.04045 (Berwick).zip'>, <DirEntry 'Berney Manuscripts--MS.2016.009'>, <DirEntry 'Worklog.pptx'>, <DirEntry 'crumbtrail.docx'>, <DirEntry 'ASSIGNMENTS.docx'>, <DirEntry 'LibraryRANotebook.ipynb'>, <DirEntry 'Worklog11-18'>, <DirEntry 'Worklog12-09.jpg'>, <DirEntry 'Transcriptions'>, <DirEntry '~$umbtrail.docx'>, <DirEntry '009_Detailed Description_(NC; 18h54_5pages).docx'>, <DirEntry 'gems.docx'>, <DirEntry '.ipynb_checkpoints'>, <DirEntry 'Definitions.docx'>, <DirEntry 'Transcription Troubleshooting.docx'>, <DirEntry 'UCLA-Clark MS.1994'>, <DirEntry 'htrc_crumbtrail.docx'>, <DirEntry '~$rc_crumbtrail.docx'>, <DirEntry 'creating_collection_instructions.docx'>, <DirEntry 'The Brocklebank riddle.docx'>, <DirEntry '004_Roles and Training of Students (nc ls JAN19_Afternoon)_NS_224PM (NS and LS complet

['002_Summary of Proposal_18h24.docx',
 'Worklog11-25.jpg',
 'RA Task List.docx',
 'To Do List.docx',
 '.DS_Store',
 'UofT_Fisher MSS.04045 (Berwick).zip',
 'Berney Manuscripts--MS.2016.009',
 'Worklog.pptx',
 'crumbtrail.docx',
 'ASSIGNMENTS.docx',
 'LibraryRANotebook.ipynb',
 'Worklog11-18',
 'Worklog12-09.jpg',
 'Transcriptions',
 '~$umbtrail.docx',
 '009_Detailed Description_(NC; 18h54_5pages).docx',
 'gems.docx',
 '.ipynb_checkpoints',
 'Definitions.docx',
 'Transcription Troubleshooting.docx',
 'UCLA-Clark MS.1994',
 'htrc_crumbtrail.docx',
 '~$rc_crumbtrail.docx',
 'creating_collection_instructions.docx',
 'The Brocklebank riddle.docx',
 '004_Roles and Training of Students (nc ls JAN19_Afternoon)_NS_224PM (NS and LS complete).docx',
 'UofT_Fisher MSS.04045 (Berwick)',
 'WorkLog Template (to be completed each Sunday by 18h please).xlsx']

In [69]:
cha = range(0)
for i in [range(10), 5, range(1,3)]:
    if type(i) is range:
        cha = chain(cha, i)
    elif type(i) is int:
        cha = chain(cha, range(i, i+1))
    
    
[i for i in cha]

TypeError: 'int' object is not iterable

In [73]:
[i for i in range(10, 11)]

[10]

In [3]:
def preProcess(dataset):
    temp = dataset.copy()
    temp[0] = temp[0].apply(lambda text: text.lower())
    temp[0] = temp[0].apply(lambda text: re.sub('<br />', ' ', text))
    temp[0] = temp[0].apply(lambda text: re.sub('[^a-z ]', '', text))
    return temp

def binBOW(dataset, counter):
    countArr = np.array(counter.transform(dataset[0]).toarray())
    countArr = (countArr != 0).astype(int)
    return pd.DataFrame(countArr, columns=counter.get_feature_names()).assign(lab=dataset[1])


def countBOW(dataset, counter):
    countArr = np.array(counter.transform(dataset[0]).toarray())
    return pd.DataFrame(countArr, columns=counter.get_feature_names()).assign(lab=dataset[1]).fillna(0)


In [4]:
imdbTrainSet = pd.read_csv(dataPath + '/IMDB-train.txt', sep='\t', lineterminator='\n', header=None)
imdbTrainSet = preProcess(imdbTrainSet)

imdbValidSet = pd.read_csv(dataPath + '/IMDB-valid.txt', sep='\t', lineterminator='\n', header=None)
imdbValidSet = preProcess(imdbValidSet)

imdbTestSet = pd.read_csv(dataPath + '/IMDB-test.txt', sep='\t', lineterminator='\n', header=None)
imdbTestSet = preProcess(imdbTestSet)

In [5]:
imdbTrainLabels = imdbTrainSet[1]
imdbValidLabels = imdbValidSet[1]
imdbTestLabels = imdbTestSet[1]
imdbTrainValidLabels = imdbTrainLabels.append(imdbValidLabels)

## Counts Dataset

In [0]:
## Create vocabulary from training set
imdbCounter = CountVectorizer(max_features=10000)

imdbCounter.fit(imdbTrainSet[0])

imdbTrainCountsBOW = countBOW(imdbTrainSet, imdbCounter)

imdbValidCountsBOW = countBOW(imdbValidSet, imdbCounter)

imdbTestCountsBOW = countBOW(imdbTestSet, imdbCounter)


KeyboardInterrupt: ignored

## TFIDF

In [6]:
## Create object to transform data to tf-idf format
tf = TfidfVectorizer(max_features=20000, ngram_range=(1, 3))
tf.fit(imdbTrainSet[0])

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=20000, min_df=1,
        ngram_range=(1, 3), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [7]:
imdbTrainTfidf = pd.DataFrame(tf.transform(imdbTrainSet[0]).toarray(), columns=tf.get_feature_names())
imdbValidTfidf = pd.DataFrame(tf.transform(imdbValidSet[0]).toarray(), columns=tf.get_feature_names())
imdbTestTfidf = pd.DataFrame(tf.transform(imdbTestSet[0]).toarray(), columns=tf.get_feature_names())

## LDA

In [0]:
lda = LatentDirichletAllocation(n_components=50, verbose=1)

In [0]:
lda.fit(imdbTrainCountsBOW)

/usr/local/lib/python3.6/dist-packages/sklearn/decomposition/online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)


KeyboardInterrupt: ignored

In [0]:
topicsLDA = pd.DataFrame(lda.components_, columns=imdbCounter.get_feature_names())

In [0]:
topicsLDAAbv = pd.DataFrame()
for index, row in topicsLDA.iterrows():
  topicsLDAAbv = topicsLDAAbv.append(pd.DataFrame([imdbCounter.get_feature_names()[i] for i in np.argsort(row)[:10]]).T)


In [0]:
imdbTrainLDA = pd.DataFrame(lda.transform(imdbTrainCountsBOW))
imdbValidLDA = pd.DataFrame(lda.transform(imdbValidCountsBOW))
imdbTestLDA = pd.DataFrame(lda.transform(imdbTestCountsBOW))


## NMF

In [8]:
nmf = NMF(50, random_state=12345, verbose=True, max_iter=50)

In [9]:
nmf.fit(tf.transform(imdbTrainSet[0]))

violation: 1.0
violation: 0.45193516442163384
violation: 0.27040749107341405
violation: 0.2269003629640173
violation: 0.19471265878363694
violation: 0.1680219115616303
violation: 0.14634321747675158
violation: 0.1280959351627554
violation: 0.11126184270875411
violation: 0.096245854798659
violation: 0.08383522909534469
violation: 0.07462237271109398
violation: 0.0674753386113177
violation: 0.06173654902399173
violation: 0.05715771895666389
violation: 0.05292050116651357
violation: 0.04825694944913173
violation: 0.043099224926664195
violation: 0.03823581422461614
violation: 0.03361318309606829
violation: 0.02956926461211621
violation: 0.026060124131232973
violation: 0.023101227964494184
violation: 0.02066925785243767
violation: 0.018725941758225124
violation: 0.01720791367852676
violation: 0.016005916251278226
violation: 0.015096982944780643
violation: 0.014408330633712637
violation: 0.01389265127773106
violation: 0.013490081225559019
violation: 0.01314065002743634
violation: 0.012846563

NMF(alpha=0.0, beta_loss='frobenius', init=None, l1_ratio=0.0, max_iter=50,
  n_components=50, random_state=12345, shuffle=False, solver='cd',
  tol=0.0001, verbose=True)

In [0]:
topicsNMF = pd.DataFrame(nmf.components_, columns=tf.get_feature_names())
topicsNMF

,abandoned,abc,abilities,ability,ability to,able,able to,about,about all,about an,...,yourself,yourself favor,youth,youve,youve got,youve seen,zero,zombie,zombies,zone
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.009385,0.000000,0.000000,0.000000,0.000000,0.000000,0.014253,0.000000
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.045005,0.000000,0.001530,...,0.019344,0.004828,0.000000,0.000000,0.000000,0.000000,0.006620,0.000000,0.000000,0.000000
2,0.027532,0.000000,0.000000,0.004748,0.017896,0.012393,0.013245,0.022217,0.000000,0.003781,...,0.004760,0.000000,0.000000,0.014837,0.004764,0.003513,0.000000,0.105521,0.070755,0.003704
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.018114,0.018354,0.084896,0.000000,0.002021,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.004758,0.000000,0.000000,0.000000,0.000000,0.022148,0.000000,0.005398,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012941,0.000000,0.000000,0.000000
5,0.000465,0.000000,0.004767,0.009759,0.000000,0.012470,0.012649,0.003373,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,0.000000,0.000000,0.000000,0.022523,0.009678,0.014298,0.011378,0.090096,0.000000,0.005454,...,0.000000,0.000000,0.019775,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.025254,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.061119,0.000000,0.005050,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,0.000000,0.000000,0.000000,0.000000,0.000000,0.001688,0.000000,0.000000,0.000000,0.002619,...,0.000000,0.000000,0.000000,0.000000,0.002463,0.000000,0.000000,0.000000,0.000000,0.000000


In [0]:
topicsNMFAbv = pd.DataFrame()
for index, row in topicsNMF.iterrows():
  topicsNMFAbv = topicsNMFAbv.append(pd.DataFrame([tf.get_feature_names()[i] for i in np.argsort(row)[:10]]).T)
topicsNMFAbv

,0,1,2,3,4,5,6,7,8,9
0,abandoned,over her,over his,over it,over the place,over years,overacting,overall,overall it,overall the
0,abandoned,quit,quite,quite as,quite bit,quite good,quite the,quite well,quote,rabbit
0,mean that,there were few,lynch,lyrics,macarthur,there were,machines,macy,lust,there wasnt
0,abandoned,population,porn,porno,portion of,portrait,portrait of,portrayal,portrayal of,portrayal of the
0,abandoned,poetry,poignant,point,point and,point in,point in the,point of,point of the,point of view
0,mean that,par with,parade,parallel,paramount,paranoia,paris,parody,parody of,part and
0,abandoned,perhaps the,period,period of,personalities,personally,persons,perspective,pg,philosophical
0,abandoned,realized that,realizes,realizes that,realizing,really,really bad,really be,really cant,really care
0,abandoned,read the,read the book,reading,reading the,reading this,reads,ready,ready for,ready to
0,abandoned,quite good,quite the,rabbit,race,racial,racism,racist,radio,rage


In [0]:
imdbTrainNMF = pd.DataFrame(nmf.transform(imdbTrainTfidf))
imdbValidNMF = pd.DataFrame(nmf.transform(imdbValidTfidf))
imdbTestNMF = pd.DataFrame(nmf.transform(imdbTestTfidf))


violation: 1.0
violation: 0.3791686611747777
violation: 0.07770474082115794
violation: 0.01753921221900882
violation: 0.006268721300653509
violation: 0.002023346850763037
violation: 0.0005111057638289919
violation: 0.00013609782327786963
violation: 4.8025682886756333e-05
Converged at iteration 9
violation: 1.0
violation: 0.3789681024894265
violation: 0.07895929869017258
violation: 0.017925624966645272
violation: 0.00649362375433549
violation: 0.002131292013459603
violation: 0.0005425433717860234
violation: 0.00014479611562148685
violation: 5.1442179713407275e-05
Converged at iteration 9
violation: 1.0
violation: 0.37805065366124024
violation: 0.07826313298815687
violation: 0.01775626504030134
violation: 0.006399301913928864
violation: 0.002089502627417968
violation: 0.0005300670430229202
violation: 0.0001412985402496063
violation: 5.018799293410001e-05
Converged at iteration 9


## TFIDF Alone Tuning

In [0]:
bestSVM = None
bestScore = 0
for n_components in np.linspace(100, 10000, 10, dtype=int):
    for nMax in range(1, 4):
        for nMin in [1, nMax]:
            ## Create object to transform data to tf-idf format
            tf = TfidfVectorizer(max_features=n_components, ngram_range=(nMin, nMax))
            tf.fit(imdbTrainSet[0])
  
            imdbTrainTfidf = pd.DataFrame(tf.transform(imdbTrainSet[0]).toarray(), columns=tf.get_feature_names())
            imdbValidTfidf = pd.DataFrame(tf.transform(imdbValidSet[0]).toarray(), columns=tf.get_feature_names())
      
            imdbTrainValidTfidf = imdbTrainTfidf.append(imdbValidTfidf)
  
            svm = LinearSVC()
      svmParams = [{'kernel': ['rbf'],
                    'C': [2**x for x in range(-5, 6)],
                    'gamma': np.linspace(0.0001, 0.999, 8)},
                   {'kernel': ['linear'],
                    'C': [2**x for x in range(-5, 6)]},
                   {'kernel': ['poly'],
                    'C': [2**x for x in range(-5, 6)],
                    'gamma': np.linspace(0.0001, 0.999, 8),
                    'coef0': [0, 1]}]
      ps = PredefinedSplit(test_fold=np.concatenate((-1*np.ones(imdbTrainTfidf.shape[0]), np.zeros(imdbValidTfidf.shape[0]))))
      gSearchSVM = GridSearchCV(svm, svmParams, verbose=3, scoring='accuracy', cv=ps, error_score=0)
      gSearchSVM.fit(imdbTrainValidTfidf, imdbTrainValidLabels)

      print(gSearchSVM.best_score_)
      if gSearchSVM.best_score_ > bestScore:
        bestScore = SearchSVM.best_score_
        bestSVM = gSearch.best_estimator_
        print('Best score yet!')
        print('n_components:')
        print(n_components)
        
        print('nMax:')
        print(nMax)
        
        print('nMin:')
        print(nMin)

In [0]:
gSearchSVM.best_estimator_

SVC(C=2, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.999, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [0]:
svm = LinearSVC(verbose=True)

svmParams = {'C': [2**x for x in range(-5, 6)]},
ps = PredefinedSplit(test_fold=np.concatenate((-1*np.ones(imdbTrainTfidf.shape[0]), np.zeros(imdbValidTfidf.shape[0]))))

imdbTrainValidTfidf = imdbTrainTfidf.append(imdbValidTfidf)

gSearchSVM = GridSearchCV(svm, svmParams, verbose=3, scoring='accuracy', cv=ps, error_score=0)
gSearchSVM.fit(imdbTrainValidTfidf, imdbTrainValidLabels)


svm.fit(imdbTrainTfidf, imdbTrainLabels)

NameError: ignored

In [0]:
svm = LinearSVC(verbose=True, C=0.4)
svm.fit(imdbTrainTfidf, imdbTrainLabels)
validPreds = svm.predict(imdbValidTfidf)
accuracy_score(imdbValidLabels, validPreds)

[LibLinear]

0.8909

In [28]:
svm = LinearSVC(verbose=True, C=0.15)
svm.fit(pd.concat((imdbTrainNMF, imdbTrainTfidf), axis=1), imdbTrainLabels)
validPreds = svm.predict(pd.concat((imdbValidNMF, imdbValidTfidf), axis=1))
accuracy_score(imdbValidLabels, validPreds)

[LibLinear]

0.8852